In [1]:
import tensorwrap as tw
import jax.numpy as jnp
from tensorwrap import nn

In [2]:
# Loading the dataset:
from tensorflow.keras import datasets
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

2023-09-08 20:51:12.144046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Preprocessing the data:

In [3]:
# Converting to tensor:
X_train, X_test = tw.convert_to_tensor(X_train, tw.float32)/255., tw.convert_to_tensor(X_test, tw.float32)/255.

In [4]:
# Splitting data between validation and train_set:
X_valid, X_train = X_train[55000:], X_train[:55000]
y_valid, y_train = y_train[55000:], y_train[:55000]

In [5]:
# Creating a class names list:
class_names = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

In [6]:
X_train_batched, y_train_batched = tw.experimental.data.Dataset(X_train).batch(128), tw.experimental.data.Dataset(y_train).batch(128)

# Building a Nueral Network:

In [7]:
# Subclassing for all features:
class Classifier(nn.Model):
    def __init__(self, name: str = "Classifier") -> None:
        super().__init__(name)
        self.flatten = nn.layers.Flatten()
        self.hidden_layers = [
            nn.layers.Dense(300),
            nn.activations.ReLU(),
            nn.layers.Dense(100),
            nn.activations.ReLU(),
            nn.layers.Dense(10)
        ]
    
    def call(self, params, inputs):
        x = self.flatten(params, inputs)
        for layer in self.hidden_layers:
            x = layer(params, x)
        return x

In [8]:
model = Classifier()
input_shape = tw.randn((1, 28, 28))
params = model.init_params(input_shape)

In [9]:
# Defining Loss Functions and optimizers and metrics:
loss_fn = nn.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = nn.optimizers.adam(learning_rate=1e-3)
state = optimizer.init(model.trainable_variables)

In [10]:
# Creating a metrics:
metrics = nn.losses.Accuracy()

In [11]:
train_state = nn.models.Train(model, loss_fn, optimizer, metrics)

In [17]:
train_state.train(X_train, y_train, 5, 128, validation_data=(X_valid, y_valid))

Epoch 1/5
429/429 [==============================>]    -    loss: 21.59295    -    metric: 80.46875    -    val_loss: 17.16390    -    val_loss: 83.14000

Epoch 2/5
429/429 [==============================>]    -    loss: 20.81426    -    metric: 75.00000    -    val_loss: 37.66552    -    val_loss: 74.80000

Epoch 3/5
429/429 [==============================>]    -    loss: 15.98670    -    metric: 76.56250    -    val_loss: 17.64361    -    val_loss: 85.64000

Epoch 4/5
429/429 [==============================>]    -    loss: 34.41030    -    metric: 82.03125    -    val_loss: 20.20567    -    val_loss: 84.24000

Epoch 5/5
429/429 [==============================>]    -    loss: 32.32131    -    metric: 79.68750    -    val_loss: 30.24812    -    val_loss: 79.74000



In [13]:
train_state.evaluate(X_test, y_test)

1/1 [==============================>]    -    loss: 22.358171463012695    -    metric: 80.17000579833984

# Creating a training loop:

In [14]:
@tw.value_and_grad
def grad_fn(params, X, y):
    pred = model(params, X)
    return loss_fn(y, pred)


@tw.function
def update(params, state, X, y):
    losses, grads = grad_fn(params, X, y)
    updates, state = optimizer.update(grads, state, params)
    params = nn.optimizers.apply_updates(params, updates)
    return params, losses, state

@tw.function
def validation(params, X, y):
    pred = model(params, X)
    return metrics(y, pred)

def train(epochs):
    global state
    for epoch in range(1, epochs+1):
        print(f"Epoch {epoch}/{epochs}")
        for index, (X, y) in enumerate(zip(X_train_batched, y_train_batched)): 
            model.trainable_variables, losses, state = update(model.trainable_variables, state, X, y)
            accuracy = metrics(y, model(model.trainable_variables, X))
            val_metrics = validation(model.trainable_variables, X_valid, y_valid)
            model.loading_animation(X_train_batched.len(), index+1, losses, accuracy, val_metric=val_metrics)
        print("\n")    

In [15]:
train(10)

Epoch 1/10
429/429 [==============================>]    -    loss: 26.36643    -    metric: 62.50000    -    val_loss: 66.50000

Epoch 2/10
429/429 [==============================>]    -    loss: 20.23694    -    metric: 75.78125    -    val_loss: 73.96000

Epoch 3/10
429/429 [==============================>]    -    loss: 27.01447    -    metric: 77.34375    -    val_loss: 78.68000

Epoch 4/10
429/429 [==============================>]    -    loss: 12.79009    -    metric: 82.03125    -    val_loss: 81.06000

Epoch 5/10
429/429 [==============================>]    -    loss: 25.15282    -    metric: 80.46875    -    val_loss: 83.25999

Epoch 6/10
429/429 [==============================>]    -    loss: 21.59295    -    metric: 80.46875    -    val_loss: 83.14000

Epoch 7/10
429/429 [==============================>]    -    loss: 20.81426    -    metric: 75.00000    -    val_loss: 74.80000

Epoch 8/10
429/429 [==============================>]    -    loss: 15.98670    -    metric: 76.56

In [16]:
model.evaluate(X_test, y_test, loss_fn=loss_fn, metric_fn=metrics)

1/1 [==============================>]    -    loss: 36.937477111816406    -    metric: 77.5